In [7]:
# Loading in libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from keras.layers import *
import keras
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.metrics import AUC
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [3]:
# Loading in data
X_raw = np.load("../input/siimisic-melanoma-resized-images/x_train_224.npy")
y_train = pd.read_csv("../input/siim-isic-melanoma-classification/train.csv")['target'].values

In [8]:
# Splitting data according to y value
X_train, y_train, X_val, y_val = train_test_split(X_raw, y_train, stratify=y_train, 
                                                  train_size=.9, random_state=1)

NameError: name 'X_raw' is not defined

In [6]:
# Freeing up some ram
del X_raw

In [ ]:
# I think it would be wise to do some data augmentation
data_gen = ImageDataGenerator(rescale=1/255., horizontal_flip=True, 
                              vertical_flip=True, rotation_range=45,
                             brightness_range=(.1,.4))

In [24]:
# Creating the architecture of the model. Using functional api and some residual layers
kernel_size = (3,3)
inp = Input(shape=(224, 224, 3))
x = SeparableConv2D(2084, kernel_size=kernel_size, activation='relu')(inp)
x = BatchNormalization()(x)
x = MaxPooling2D()(x)
x = SeparableConv2D(1042, kernel_size=kernel_size, activation='relu')(x)
x = BatchNormalization()(x)
y = MaxPooling2D()(x)
x = SeparableConv2D(521, kernel_size=kernel_size, activation='relu', padding='same')(y)
x = BatchNormalization()(x)
x = SeparableConv2D(210, kernel_size=kernel_size, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = SeparableConv2D(103, kernel_size=kernel_size, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
y = SeparableConv2D(103, kernel_size=kernel_size, activation='relu', padding='same')(y)
y = BatchNormalization()(y)
z = Add()([x,y])
z = GlobalAveragePooling2D()(z)
output = Dense(1, activation='sigmoid')(z)

model = keras.Model(inputs=inp, outputs=output)

In [25]:
# Compiling the model
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
separable_conv2d_47 (SeparableC (None, 222, 222, 208 8363        input_15[0][0]                   
__________________________________________________________________________________________________
batch_normalization_16 (BatchNo (None, 222, 222, 208 8336        separable_conv2d_47[0][0]        
__________________________________________________________________________________________________
max_pooling2d_25 (MaxPooling2D) (None, 111, 111, 208 0           batch_normalization_16[0][0]     
____________________________________________________________________________________________

In [ ]:
# Compiling
model.compile(optimizer=Adam(.001), loss='binary_crossentropy', metrics=[AUC(name='auc')])

In [ ]:
# Callbacks
mcp_save = ModelCheckpoint("best_weights.hdf5", monitor='val_loss', save_best_only=True)

In [ ]:
# Training baby
model.fit(data_gen.flow(X_train, y_train), val_data=(X_val, y_val), 
         epochs=10, batch_size=40, steps_per_epoch=len(X_train)//40,
         verbose=2)